# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122144e+02     1.581804e+00
 * time: 0.10100913047790527
     1     1.168604e+01     1.013555e+00
 * time: 1.1007311344146729
     2    -1.130356e+01     1.073502e+00
 * time: 1.2745511531829834
     3    -3.364209e+01     7.915281e-01
 * time: 1.4854390621185303
     4    -4.734110e+01     5.960793e-01
 * time: 1.633932113647461
     5    -5.667382e+01     2.120848e-01
 * time: 1.798760175704956
     6    -5.973279e+01     9.195490e-02
 * time: 1.9264872074127197
     7    -6.080512e+01     5.949828e-02
 * time: 2.0445690155029297
     8    -6.120767e+01     7.935906e-02
 * time: 2.1739771366119385
     9    -6.155618e+01     3.920762e-02
 * time: 2.2924721240997314
    10    -6.177272e+01     2.724176e-02
 * time: 2.411177158355713
    11    -6.193265e+01     2.506273e-02
 * time: 2.5367300510406494
    12    -6.200530e+01     2.077603e-02
 * time: 2.652858018875122
    13    -6.207400e+01     1.451971e-02
 * time: 2.7808101177215

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557682
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666462013
